In [13]:
import json
import spacy
import random
from pathlib import Path
from spacy.util import minibatch, compounding

In [16]:
def parseJSONToSpacySmiple(trainingSet):

    f = open(trainingSet, 'r')
    dataset = json.load(f)
    
    #print(dataset[0]['paragraphs'][0]['sentences'][0]['tokens'][0]['dep'])
        
    #The error here due to relative positions in head tag
    #Calculate the proper head by adding head to id and replacing head 0 (root) with id
    
    TRAIN_DATA = []
    
    for d in dataset:
        for p in d['paragraphs']:
            for se in p['sentences']:
                headsDepDict = {}
                heads = []
                deps = []
                text = ''
                for t in se['tokens']:
                    h = t['id'] + t['head']
                    heads.append(h)
                    deps.append(t['dep'])
                    text += t['orth'] + ' '
                headsDepDict['heads'] = heads
                headsDepDict['deps'] = deps
                finalSentTuple = (text, headsDepDict)
                TRAIN_DATA.append(finalSentTuple)    
    f.close()
    
    return TRAIN_DATA

ROOT


In [19]:
def trainParser(TRAIN_DATA, model=None, outputDir=None)

    if model is not None:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("sv")  # create blank Language class
        print("Created blank 'sv' model")
    
    if "parser" not in nlp.pipe_names:
        parser = nlp.create_pipe("parser")
        nlp.add_pipe(parser, first=True)
        
    else:
        parser = nlp.get_pipe("parser")
    
    for _, annotations in TRAIN_DATA:
        for dep in annotations.get("deps", []):
            parser.add_label(dep)
    
    pipe_exceptions = ["parser"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
            
        for itn in range(5):
            random.shuffle(TRAIN_DATA)
            losses = {}
             #batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, losses=losses)
            print("Losses", losses)
        
    print('finally done')
    
    if outputDir is not None:
        output_dir = Path(outputDir)
        if not outputDir.exists():
            outputDir.mkdir()
        nlp.to_disk(outputDir)
        print("Saved model to ", outputDir)



Losses {'parser': 6914.239720644246}
Losses {'parser': 5645.849165849023}
Losses {'parser': 4799.7300084911}
Losses {'parser': 4373.385860583101}
Losses {'parser': 4059.1224395254317}
finally done


In [20]:
TRAIN_DATA = parseJSONToSpacySmiple('pathToTrainingSet') #swedish-lines json
trainParser(TRAIN_DATA, 'pathToModel', 'pathToOutputDir')